In [ ]:
# Change runtime type to T4 GPU

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-13odo_eb
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-13odo_eb
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 5741c522547756ac4bb7a16df32106a15efb8a57
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10742 sha256=8f76f24c8ef76f4567737825c96eef1bce4cf6e61dca70f1aea8c5c7f7a278f6
  Stored in directory: /tmp/pip-ephem-wheel-cache-4e8_l01y/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter


In [ ]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmp6o655qz4".


In [ ]:
%%cuda

#include <iostream>
#include <cstdlib>
#include <ctime>

// Kernel function for vector addition
__global__ void vectorAdd(int *a, int *b, int *c, int n) {
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    if (index < n) {
        c[index] = a[index] + b[index];
    }
}

int main() {
    int n = 1000; // Size of the vectors
    int *a, *b, *c; // Host vectors
    int *d_a, *d_b, *d_c; // Device vectors
    int size = n * sizeof(int);

    // Allocate memory for host vectors
    a = (int*)malloc(size);
    b = (int*)malloc(size);
    c = (int*)malloc(size);

    // Initialize host vectors with random values
    srand(time(NULL));
    std::cout << "Vector a:" << std::endl;
    for (int i = 0; i < n; ++i) {
        a[i] = rand() % 100;
        std::cout << a[i] << " ";
    }
    std::cout << std::endl;

    std::cout << "Vector b:" << std::endl;
    for (int i = 0; i < n; ++i) {
        b[i] = rand() % 100;
        std::cout << b[i] << " ";
    }
    std::cout << std::endl;

    // Allocate memory for device vectors
    cudaMalloc(&d_a, size);
    cudaMalloc(&d_b, size);
    cudaMalloc(&d_c, size);

    // Copy host vectors to device
    cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

    // Launch kernel for vector addition
    int blockSize = 256;
    int gridSize = (n + blockSize - 1) / blockSize;
    vectorAdd<<<gridSize, blockSize>>>(d_a, d_b, d_c, n);

    // Copy result back to host
    cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);

    // Print result vector c
    std::cout << "Result vector c:" << std::endl;
    for (int i = 0; i < n; ++i) {
        std::cout << c[i] << " ";
    }
    std::cout << std::endl;

    // Free device memory
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    // Free host memory
    free(a);
    free(b);
    free(c);

    return 0;
}

Vector a:
37 33 5 72 64 98 49 88 32 96 10 41 13 56 81 70 9 19 28 66 65 82 16 32 1 78 87 50 26 46 3 64 31 61 36 48 11 86 36 43 34 47 36 47 55 18 69 16 89 49 82 54 32 98 86 33 77 25 35 3 24 39 19 55 52 56 55 63 94 92 7 80 91 95 27 46 13 49 14 3 98 96 9 30 94 48 16 23 73 51 79 97 42 98 5 94 6 60 10 52 4 69 32 47 64 12 45 78 61 59 33 59 55 42 42 2 42 58 77 16 61 56 65 56 7 22 2 13 35 12 66 39 81 50 87 46 62 32 76 23 44 9 35 99 3 77 53 46 87 31 14 0 39 79 56 46 2 59 12 37 71 78 28 5 80 67 51 43 52 79 18 96 88 53 95 43 82 49 89 21 32 3 22 71 35 78 18 89 89 30 26 13 60 6 18 40 74 21 35 26 0 54 22 88 59 69 83 42 70 73 63 2 28 37 74 15 68 44 4 57 74 82 70 86 89 88 78 15 61 14 93 61 68 67 1 79 36 85 21 59 10 37 61 38 74 87 6 94 31 10 52 57 45 22 95 86 63 74 1 24 40 46 86 60 13 39 39 49 24 61 60 34 50 22 25 76 61 31 71 45 93 23 54 38 97 50 24 12 76 77 37 16 23 75 76 36 66 15 38 91 28 98 77 30 72 54 7 86 37 30 31 31 5 85 21 54 87 98 67 63 75 56 79 51 83 7 39 49 75 29 92 55 80 22 86 52 76 45 38 14 

In [ ]:
%%cuda

#include <iostream>
#include <cstdlib>
#include <ctime>

#define N 64 // Reduced size of the square matrices

// Kernel function for matrix multiplication
__global__ void matrixMul(int *a, int *b, int *c, int n) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int sum = 0;
    if (row < n && col < n) {
        for (int i = 0; i < n; ++i) {
            sum += a[row * n + i] * b[i * n + col];
        }
        c[row * n + col] = sum;
    }
}

int main() {
    int *a, *b, *c; // Host matrices
    int *d_a, *d_b, *d_c; // Device matrices
    int size = N * N * sizeof(int);

    // Allocate memory for host matrices
    a = (int*)malloc(size);
    b = (int*)malloc(size);
    c = (int*)malloc(size);

    // Initialize host matrices with random values
    srand(time(NULL));
    for (int i = 0; i < N * N; ++i) {
        a[i] = rand() % 100;
        b[i] = rand() % 100;
    }

    // Allocate memory for device matrices
    cudaMalloc(&d_a, size);
    cudaMalloc(&d_b, size);
    cudaMalloc(&d_c, size);

    // Copy host matrices to device
    cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

    // Launch kernel for matrix multiplication
    dim3 threadsPerBlock(16, 16);
    dim3 numBlocks((N + threadsPerBlock.x - 1) / threadsPerBlock.x, (N + threadsPerBlock.y - 1) / threadsPerBlock.y);
    matrixMul<<<numBlocks, threadsPerBlock>>>(d_a, d_b, d_c, N);

    // Copy result back to host
    cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);

    // Print result
    std::cout << "Matrix A:" << std::endl;
    for (int i = 0; i < N * N; ++i) {
        std::cout << a[i] << " ";
        if ((i + 1) % N == 0) {
            std::cout << std::endl;
        }
    }

    std::cout << std::endl << "Matrix B:" << std::endl;
    for (int i = 0; i < N * N; ++i) {
        std::cout << b[i] << " ";
        if ((i + 1) % N == 0) {
            std::cout << std::endl;
        }
    }

    std::cout << std::endl << "Result Matrix C:" << std::endl;
    for (int i = 0; i < N * N; ++i) {
        std::cout << c[i] << " ";
        if ((i + 1) % N == 0) {
            std::cout << std::endl;
        }
    }

    // Free device memory
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    // Free host memory
    free(a);
    free(b);
    free(c);

    return 0;
}

Matrix A:
18 48 24 29 93 8 11 18 57 13 15 44 38 58 67 24 69 72 40 59 75 40 66 84 19 64 37 49 41 65 47 46 65 62 72 92 94 35 43 35 73 96 60 38 6 14 97 74 31 20 63 3 97 69 62 3 56 97 49 23 91 13 50 2 
28 88 87 0 82 77 78 35 83 55 20 40 20 3 93 55 33 65 87 9 39 18 18 55 39 86 93 27 32 87 88 99 86 94 81 95 41 41 72 12 60 26 30 71 84 40 1 46 7 12 95 13 26 99 51 13 62 69 93 7 98 1 42 38 
91 41 63 74 13 39 37 80 91 96 41 65 6 91 18 1 84 48 1 88 0 30 79 41 11 11 44 44 96 50 68 76 45 52 52 12 20 42 5 27 24 73 15 22 4 30 27 30 83 23 27 40 30 52 69 9 69 65 52 0 63 33 89 14 
50 12 77 29 90 47 64 4 91 23 8 51 6 13 52 71 69 84 91 5 47 92 13 49 69 99 48 0 88 53 45 95 64 12 44 41 74 7 34 44 81 85 47 87 51 27 68 62 5 44 43 93 22 10 61 64 50 75 34 84 6 85 38 63 
21 94 92 87 92 23 3 47 88 3 88 12 52 72 45 34 42 35 72 1 71 69 15 81 42 77 72 28 39 67 88 2 34 17 64 22 98 11 11 16 28 87 35 77 67 10 98 86 54 42 77 55 7 40 62 75 53 43 29 66 17 4 41 51 
97 66 57 1 43 86 1 0 75 47 64 5 90 49 51 30 46 74 70 72 20 